In [1]:
import sys
sys.path.append("../src")
from utils.common.pathManager import FilePath
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from cv2 import imread
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score
xbd_path = FilePath("/media/mrtc101/Martín SSD/tesis/datasets/xBD")

In [ ]:

from email.headerregistry import DateHeader
import torch
from models.trainable_model import TrainModel
from utils.datasets.train_dataset import TrainDataset


loader: DateHeader
model: TrainModel
device : torch.device
metric_dir: str

k: int = 100
dataset: TrainDataset = loader.dataset
n_class = 5
model.eval()
# columns=["tp","fp","fn","tn"]
patch_conf_mtrx_dict = defaultdict(lambda: torch.zeros(size=(n_class, 4)))
for i in trange(0, int(len(dataset)/16)):
    dis_id, tile_id, patch_dict = dataset.get_by_id(i)
    for patch_id, patch in patch_dict.items():
        x_pre = patch['pre_img'].to(device=device)
        x_post = patch['post_img'].to(device=device)
        y_cls = patch['dmg_mask'].to(device=device)
        bin_true_tensor = model.make_binary(y_cls, [0, 1, 2, 3, 4])
        logit_masks = model(x_pre.unsqueeze(0), x_post.unsqueeze(0))

        for th in torch.arange(0, 1+(1/k), 1/k):
            bin_pred_tensor = model.compute_binary(logit_masks[2], th)
            bin_pred_tensor = bin_pred_tensor.squeeze(0)
            conf_matrices = MatrixComputer.compute_bin_matrices_px(
                    bin_pred_tensor, bin_true_tensor)
            key = round(float(th), 3)
            patch_conf_mtrx_dict[key] += conf_matrices.to(device='cpu')


In [ ]:
roc_curve: tuple = MetricComputer.compute_ROC(patch_conf_mtrx_dict)
plot_roc_curves("pixel", roc_curve, metric_dir)
pr_curve: tuple = MetricComputer.compute_PR(patch_conf_mtrx_dict)
plot_pr_curves("pixel", pr_curve, metric_dir)
